In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#ADD ANY IMPORTS HERE
import pandas as pd
import numpy as np
import math 
import random
import time
from collections import Counter
from pathlib import Path
import plotly.graph_objects as go

In [3]:
def PreprocessData():
  # READ THE DATA IN FROM CSV FOR AND TRANSFORMING IT TO MATRIX COLUMNS ARE BUSINESS IDS AND ROWS ARE USERS
  df = pd.read_csv('/content/drive/My Drive/CS419 FINAL PROJECT GROUP 3/Yelp/ratings.csv', sep=',')
  
  # drop the date since we aren't using it 
  df.drop(["Date"], axis=1, inplace=True)
  df.sort_values(by=["User ID"], inplace=True)
  df = df.reset_index(drop=True)

  counter = Counter(df["User ID"].values)
  test_df = pd.DataFrame()
  # SEPARATE TRAINING DATA AND TEST DATA 
  for user in counter:
      if counter[user] >= 2:
          get_this_many = math.floor(counter[user] * .8)
          sample_df = df[df["User ID"] == user].sample(n=get_this_many)
          df = df.drop(sample_df.index)
          test_df = pd.concat([test_df, sample_df])
          if len(test_df) >= 6748:
              break

  train = df.pivot_table(index="User ID", values="Rating", columns="Business ID")
  test = test_df.pivot_table(index="User ID", values="Rating", columns="Business ID")
  
  # MovieLens Data 
  mldf = pd.read_csv('/content/drive/My Drive/CS419 FINAL PROJECT GROUP 3/MovieLens/u1.base', sep='\t', names=["User ID", "Item ID", "Rating", "Timestamp"])
  mldf.drop(["Timestamp"], axis=1, inplace=True)
  ML_Test = pd.read_csv('/content/drive/My Drive/CS419 FINAL PROJECT GROUP 3/MovieLens/u1.test', sep='\t', names=["User ID", "Item ID", "Rating", "Timestamp"])
  ML_Test.drop(["Timestamp"], axis=1, inplace=True)
  trainML = mldf.pivot_table(index="User ID", values="Rating", columns="Item ID")
  testML = ML_Test.pivot_table(index="User ID", values="Rating", columns="Item ID")

  return train, test, trainML, testML

In [4]:
""" Jesus Garcia 7/21/20"""
def BaselineStatisticalMethod(Train, Test):
  itm_id_train = Train.columns.values.tolist() # A list of item ID numbers to verify that the rating we observe belongs to a specified item or business ID
  itm_id_test = Test.columns.values.tolist()
  usr_id_train = Train.index.values.tolist()
  usr_id_test = Test.index.values.tolist()

  Train = Train.to_numpy() # Convert Train-setDataframe to Numpy Array
  Test = Test.to_numpy()# Convert Test-set Dataframe to Numpy Array

  i, j = Train.shape # Training Dataset Dimension RowxCol
  ti, tj = Test.shape # Test Dataset Dimension RowxCol
  u = np.nanmean(Train) # Training Dataset observed rating mean

  usr_means = np.nanmean(Train, axis=1,dtype=float) # A list of the avg of ratings for each user (mean(bu's))
  itm_means = np.nanmean(Train, axis=0,dtype=float) # A list of the avg of each item's ratings (mean(bi's))
  usr_dev = [rt - u for rt in usr_means] # A list of deviations from a user's average to the training set observed ratings average
  itm_dev = [rt - u for rt in itm_means] # A list of deviations from a items's average to the training set observed ratings average

  matrix = BuildPredictionMatrix(Test, usr_dev, itm_dev, itm_id_train, itm_id_test, usr_id_train, usr_id_test, u)
  MAE = (CalculateMAE(Test, matrix, u))

  return MAE


def BuildPredictionMatrix(TestData, usr_dev, itm_dev, itm_id_train, itm_id_test, usr_id_train, usr_id_test, u):
  """ This method aims to take the training data
      (bu and bi deviations and global mean)
      and create a matrix for predictions that 
      will be later evaluated for its similarity 
      to the test set.

      NOTE:The only reason the test data is 
      passed in is to observe the position 
      of where to make the prediction!
      
      """
  ti, tj = TestData.shape
  prediction_matrix = np.empty((ti, tj))
  prediction_matrix[:] = np.NaN
  
  for row in range(0,ti):
    for col in range(0, tj):   
      if(math.isnan(TestData[row][col]) == False): # TestSet is utilized to observe position of where a prediction is needed
          for x in range(0, len(itm_id_train)): # This loop checks the training set's item ids to find the correct deviation for the item on the test set
            if(itm_id_test[col] == itm_id_train[x]):
              for y in range(0, len(usr_id_train)): # This loop checks the training set's user ids to find the correct deviation for the user on the test set
                if(usr_id_test[row] == usr_id_train[y]):
                  bi = itm_dev[x]
                  bu = usr_dev[y]
                  prediction_matrix[row][col] = u + bu + bi # Prediction is made based on the sum of the training set mean, bu & bi deviations to each respective user and item

                  # if(prediction_matrix[row][col] > 5):
                  #   prediction_matrix[row][col] = 5
                  # elif(prediction_matrix[row][col] < 0):
                  #   prediction_matrix[row][col] = 0
  return prediction_matrix


def CalculateMAE(Actual, Predicted, u):
    ti, tj = Actual.shape
    n = 0
    sum = 0

    for row in range(0,ti):
      for col in range(0, tj):   
        if(math.isnan(Actual[row][col]) == False and math.isnan(Predicted[row][col]) == False):
          sum = sum + abs((Actual[row][col] - Predicted[row][col]))
          n += 1

    return round((sum/n),3)

In [5]:
""" Jesus Garcia 7/25/20 """
def BaselineGradientDescent(Train, Test, DatasetName, iterations):

  itm_id_train = Train.columns.values.tolist() # A list of item ID numbers to verify that the rating we observe belongs to a specified item or business ID
  itm_id_test = Test.columns.values.tolist()
  usr_id_train = Train.index.values.tolist() # A list of user ID numbers to verify that the rating we observe belongs to a specified user ID
  usr_id_test = Test.index.values.tolist()

  Train = Train.to_numpy() # Convert Train-setDataframe to Numpy Array
  Test = Test.to_numpy() # Convert Test-set Dataframe to Numpy Array
  i, j = Train.shape # Dimensions of Training Data Row x Col

  # info = np.empty((4, 5))
  # info[:] = np.NaN
  # info[0][0] = 3
  # info[0][1] = 2
  # info[0][2] = 5
  # info[0][3] = 3
  # info[1][1] = 1
  # info[1][3] = 5
  # info[1][4] = 2
  # info[2][0] = 3
  # info[2][2] = 2
  # info[2][3] = 2
  # info[3][0] = 1
  # info[3][2] = 3
  # info[3][4] = 2
  # i, j = info.shape

  u = round(np.nanmean(Train), 3)
  bu_lst = [random.randrange(0, 5, 1) for i in range(i)]
  bi_lst = [random.randrange(0, 5, 1) for i in range(j)]

  usr_dev = [rt for rt in bu_lst] # Initial Bu
  itm_dev = [rt for rt in bi_lst] # Initial Bi
  iter_list = []
  iter_list.append(0)
  COST_LIST = []
  learning_rate = 0.02

  for epoch in range(0,iterations,1):
    if(epoch == 0):
      COST_LIST.append(CalculateLossFunction(Train, usr_dev, itm_dev)) # Store data
    
    for row in range(i):
      for col in range(j):
        if(math.isnan(Train[row][col]) == False):
          eui = (Train[row][col] - u - usr_dev[row] - itm_dev[col])
          usr_dev[row] = usr_dev[row] - learning_rate * (-2 * eui + (2 * 0.25 * usr_dev[row])) # Check without absolute value
          itm_dev[col] = itm_dev[col] - learning_rate * (-2 * eui + (2 * 0.25 * itm_dev[col]))

    PredictionMatrix = BuildPredictionMatrix(Test, usr_dev, itm_dev, itm_id_train, itm_id_test, usr_id_train, usr_id_test, u )
    MAE = CalculateMAE(Test, PredictionMatrix, u)
    # Plot val of loss func against differnt epoch

    if(epoch == 0 or epoch == 4 or epoch == 9 or epoch == 49 or epoch == 99):
      # print("The MAE for a training instance with " + str(epoch + 1) + " epoch(s) is: " + str(MAE))
      iter_list.append(epoch + 1)
      COST_LIST.append(CalculateLossFunction(Train, usr_dev, itm_dev)) # Store data

      if epoch == 99:
        
        GraphMAES(COST_LIST, iter_list, DatasetName)

  return MAE

def GraphMAES(COSTS, EPOCHS, Dname):
  COSTS = np.array(COSTS, dtype=float)
  EPOCHS = np.array(EPOCHS, dtype=float)
  # plt.plot(EPOCHS, MAES, 'o')
 

  fig = go.Figure(
      data=go.Scatter(
      x= EPOCHS,
      y= COSTS,
      marker=dict(color="crimson", size=12),
      name=Dname,
  ))

  fig.update_layout(title="LOSS FUNCTION COST OVER EPOCHS FOR " + Dname + " DATASET",
                    xaxis_title="EPOCH",
                    yaxis_title="COST")

  fig.show()


def CalculateLossFunction(dataset, bu_dev_list, bi_dev_list):
  i, j = dataset.shape
  bu_dev_list = [pow(bu_dev, 2) for bu_dev in bu_dev_list] 
  bi_dev_list = [pow(bi_dev, 2) for bi_dev in bi_dev_list] 
  u = round(np.nanmean(dataset), 3)
  hyper_parameter = 0.25 * (sum(bu_dev_list) + sum(bi_dev_list))
  add = 0

  for row in range(i):
    for col in range(j):
      if(math.isnan(dataset[row][col]) == False):
        eui = math.pow((dataset[row][col] - u - bu_dev_list[row] - bi_dev_list[col]) , 2)
        add += eui

  return add + hyper_parameter 

In [ ]:
def main():
    YelpTrain, YelpTest, TrainML, TestML = PreprocessData()

    df_yelp_train = pd.read_csv('/content/drive/My Drive/CS419 FINAL PROJECT GROUP 3/Yelp/formatted_yelp_train.tsv', sep='\t', index_col="User ID") # This should be all you need
    df_yelp_test = pd.read_csv('/content/drive/My Drive/CS419 FINAL PROJECT GROUP 3/Yelp/formatted_yelp_test.tsv', sep='\t', index_col="User ID") # This should be all

    df_yelp_train.head()
    #Utilize the time package to keep track of the duration of your algorithm implementations
    
    # BASELINE STATISTICAL COMPUTATION
    start = time.time()
    MLMAE = BaselineStatisticalMethod(TrainML, TestML)
    YPMAE = BaselineStatisticalMethod(df_yelp_train, df_yelp_test)
    end = time.time()
    print("The MAE for the baseline prediction model using statistical computation is: " + str(MLMAE) + " for the MovieLens Dataset")
    print("The MAE for the baseline prediction model using statistical computation is: " + str(YPMAE) + " for the Yelp Dataset\n")
    print("The Statistical Prediction Baseline Model took " + str(round(end - start, 3)) + " seconds to run\n \n")

    # BASELINE GRADIENT DESCENT
    print("\n" + "Initializing the training of the baseline model through gradient descent\n")
    # learn_rates = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04]
    # mae_out = np.empty((2, 8), dtype=float)
    epochs = [1, 5, 10, 50, 100]
    n = 0
    epoch_maes = np.empty((2, 5)) # Row 1 is MOVIELENS data ROW 2 is Yelp data
    start2 = time.time()
    while n <= 4:
      epoch_maes[0][n] = BaselineGradientDescent(TrainML, TestML, "MOVIELENS", epochs[n])
      epoch_maes[1][n] = BaselineGradientDescent(df_yelp_train, df_yelp_test, "YELP", epochs[n])
      print("One instance of the training cycle has finished with " + str(epochs[n]) + " epoch(s)" )
      n +=1
    
    print("\n")
    print("\n")
    print("\n")
    
    end2 = time.time()
    print("\nThe Gradient Descent Baseline Model took " + str(round(end2 - start2, 3)) + " seconds to run\n")
    print("For the MovieLens Dataset, the following MAE's were recorded for the specified number of epochs for that instance:\n")
    for i in range(n):
      print("For an instance with " + str(epochs[i]) + " epoch(s) the MAE is: " + str(epoch_maes[0][i]))
    print()

    print("For the Yelp Dataset, the following MAE's were recorded for the specified number of epochs for that instance:\n")
    for i in range(n):
      print("For an instance with " + str(epochs[i]) + " epoch(s) the MAE is: " + str(epoch_maes[1][i]))
    # np.savetxt("GRADIENTDESCENTMAES.txt", np.array(mae_out), fmt="%s")  # This saved the maes for the learning rates(GRADIENTDESCENTMAES.txt) in the list "learn_rates" and I chose the min mae for the learn rate

    # ITEM BASED SIMILARITY
    k_values = np.array([1, 5, 10, 50, 100])

    print("Movie Lens Data")
    for i in range(len(k_values)):
        print("Item-based similarity model for k = " + str(k_values[i]) + ":")
        start_time = datetime.datetime.now()
        item_based_similarity_model(k_values[i], MLTrain.T, MLTest.T)
        elapsed_time = datetime.datetime.now() - start_time
        print("Elapsed time: " + str(elapsed_time) + "\n")
    
    print("Yelp Data")
    for i in range(len(k_values)):
        print("Item-based similarity model for k = " + str(k_values[i]) + ":")
        start_time = datetime.datetime.now()
        item_based_similarity_model(k_values[i], df_yelp_train.T, df_yelp_test.T)
        elapsed_time = datetime.datetime.now() - start_time
        print("Elapsed time: " + str(elapsed_time) + "\n")
    

if __name__ == '__main__':
    main()

The MAE for the baseline prediction model using statistical computation is: 0.771 for the MovieLens Dataset
The MAE for the baseline prediction model using statistical computation is: 1.071 for the Yelp Dataset

The Statistical Prediction Baseline Model took 11.008 seconds to run
 


Initializing the training of the baseline model through gradient descent

One instance of the training cycle has finished with 1 epoch(s)
One instance of the training cycle has finished with 5 epoch(s)
One instance of the training cycle has finished with 10 epoch(s)
